# Importando Série Histórica do BTC

In [94]:
import json
import requests
import pandas as pd

# insert your API key here
API_KEY = '2DW9AJS0I2CjieLcF9nhVeDVU79'

# make API request
res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc', params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
btc = pd.read_json(res.text, convert_dates=['t'])
btc.rename(columns = {"t" : "Data", "o" : "BTC"}, inplace = True)
pd.to_datetime(btc["Data"], format='%d %b %Y')
btc = btc.from_records(btc["BTC"].tolist()).join(btc).drop("BTC", axis = 1).set_index("Data")
btc.rename(columns = {"o" : "Abertura", "h" : "Máxima", "l" : "Mínima", "c" : "Fechamento"}, inplace = True)
btc.to_excel("serie_bitcoin.xlsx")
btc.head()

,Fechamento,Máxima,Mínima,Abertura
Data,,,,
2010-07-17,0.049510,0.04951,0.04951,0.04951
2010-07-18,0.085840,0.08585,0.05941,0.05941
2010-07-19,0.080800,0.09307,0.07855,0.09090
2010-07-20,0.074733,0.08181,0.07426,0.08181
2010-07-21,0.079210,0.07921,0.06634,0.07425


# Importando Indicadores

In [95]:
df = pd.DataFrame()

In [3]:
indicadores_luis = pd.read_excel("../raw_data/Indicadores Luis.xlsx")
indicadores_luis.index = indicadores_luis["Data"]
indicadores_luis.drop("Data", axis = 1, inplace = True)
indicadores_luis.head()

,MayerMutiple,BPT,DeltaGradient140,DeltaGradient28,MVRV,MVRVZ,Thermocap
Data,,,,,,,
2010-07-18,NaN,NaN,NaN,NaN,146.038332,2.518053,5.025170e-07
2010-07-19,NaN,NaN,NaN,NaN,62.565130,2.097027,4.708351e-07
2010-07-20,NaN,NaN,NaN,NaN,45.106630,1.651545,4.336639e-07
2010-07-21,NaN,NaN,NaN,NaN,38.357560,1.927019,4.577077e-07
2010-07-22,NaN,NaN,NaN,NaN,25.132348,0.885502,3.221589e-07


In [97]:
indicadores_samu = pd.read_excel("../raw_data/Indicadores Samuel.xlsx")
indicadores_samu.index = indicadores_samu["Data"]
indicadores_samu.drop("Data", axis = 1, inplace = True)

df = indicadores_samu.join(indicadores_luis)
indicadores_samu.head()

,Daily Inflation Ratio,Annual Inflation Rate,Active Addresses,Hash Rate 30 Day,Median Transaction Size
Data,,,,,
2009-01-03,NaN,NaN,0,NaN,NaN
2009-01-04,NaN,NaN,0,NaN,NaN
2009-01-05,NaN,NaN,0,NaN,NaN
2009-01-06,NaN,NaN,0,NaN,NaN
2009-01-07,NaN,NaN,0,NaN,NaN


In [98]:
indicadores_macro = pd.read_excel("../raw_data/Indicadores Macro.xlsx")
indicadores_macro.dropna(inplace = True)
indicadores_macro.set_index("Data", drop = True, inplace = True)
df = df.join(indicadores_macro)
indicadores_macro.head()

,SPX,DXY,GLD,10YIELD
Data,,,,
2011-02-01,0.016694,-0.008491,0.007161,0.009865
2011-02-02,-0.002723,0.001168,-0.002676,0.021906
2011-02-03,0.002354,0.007646,0.013415,0.018830
2011-02-04,0.002884,0.003730,-0.004085,0.034973
2011-02-07,0.006240,-0.000128,0.000152,0.007418


# Importando do Google Trends

In [99]:
def trends_import(palavra):
    import pytrends 
    from pytrends.request import TrendReq
    import pandas as pd
    import datetime
    
    pytrends = TrendReq(hl='en-US', tz=360)
    # build payload
    kw_list = [palavra] # list of keywords to get data 
    pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m')
    data = pytrends.get_historical_interest(kw_list,
                                            year_start=2009, month_start=1, day_start=1, hour_start=0,
                                            year_end=2021, month_end=12, day_end=30, hour_end=0,
                                            cat=0, sleep=0, frequency = "daily")
    return data

In [100]:
trends = trends_import("Bitcoin")
trends.drop("isPartial", axis = 1, inplace = True)
trends.columns = ["Pesquisas"]
df = df.join(trends)
trends.head()

,Pesquisas
date,
2009-01-01,0
2009-01-02,0
2009-01-03,0
2009-01-04,0
2009-01-05,0


# Importando do Glassnode

In [101]:
def glassnode_import(indicador):

    import json
    import requests
    import pandas as pd

    # insert your API key here
    API_KEY = '2DW9AJS0I2CjieLcF9nhVeDVU79'

    # make API request
    res = requests.get(f'https://api.glassnode.com/v1/metrics/indicators/{indicador}', params={'a': 'BTC', 'api_key': API_KEY})

    # convert to pandas dataframe
    df = pd.read_json(res.text, convert_dates=['t'])
    df.rename(columns = {"t" : "date", "v" : indicador}, inplace = True)
    df.index = df["date"]
    df.drop("date", axis = 1, inplace = True)
    return df

In [102]:
sopr = glassnode_import("sopr")
unrealized_profit = glassnode_import("net_unrealized_profit_loss")
cdd = glassnode_import("cdd")

In [103]:
stock_to_flow_ratio = glassnode_import("stock_to_flow_ratio")
stock_to_flow_ratio.reset_index(inplace = True)
stock_to_flow_ratio = stock_to_flow_ratio.from_records(stock_to_flow_ratio["o"].tolist()).join(stock_to_flow_ratio).drop("o", axis = 1)
stock_to_flow_ratio.set_index("date", inplace = True)

stock_to_flow_ratio["DELTA_STF"] = (stock_to_flow_ratio["price"]/stock_to_flow_ratio["ratio"])-1
stock_to_flow_ratio.drop(["ratio", "price"], axis = 1, inplace = True)

In [104]:
# Adicionando todos os indicadores na base
nomes = [sopr, unrealized_profit, cdd, stock_to_flow_ratio]
for i in nomes: df = df.join(i)

In [105]:
df

,Daily Inflation Ratio,Annual Inflation Rate,Active Addresses,Hash Rate 30 Day,Median Transaction Size,MayerMutiple,BPT,DeltaGradient140,DeltaGradient28,MVRV,...,SPX,DXY,GLD,10YIELD,Pesquisas,sopr,net_unrealized_profit_loss,cdd,daysTillHalving,DELTA_STF
2009-01-03,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-13,0.004544,1.658383,834925,2.021100e+08,70.004088,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000379,NaN,NaN,624.0,-0.767433
2022-08-14,0.004347,1.586729,811197,2.014597e+08,65.429275,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.998799,NaN,NaN,623.0,-0.768921
2022-08-15,0.004739,1.729810,922140,2.010798e+08,85.083070,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.998527,NaN,NaN,622.0,-0.771187
2022-08-16,0.004837,1.765514,1020819,2.019993e+08,83.585980,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.999409,NaN,NaN,621.0,-0.773824


# Exportando Indicadores

In [106]:
# Nomes antigos das variáveis
dados = df.copy()
dados.columns.value_counts()

Pesquisas                     1
MVRV                          1
sopr                          1
Active Addresses              1
MayerMutiple                  1
Annual Inflation Rate         1
GLD                           1
DXY                           1
Hash Rate 30 Day              1
BPT                           1
Thermocap                     1
net_unrealized_profit_loss    1
MVRVZ                         1
cdd                           1
daysTillHalving               1
DeltaGradient140              1
Median Transaction Size       1
Daily Inflation Ratio         1
DeltaGradient28               1
10YIELD                       1
DELTA_STF                     1
SPX                           1
dtype: int64

In [107]:
# Renomeando indicadores conforme documentação (https://docs.google.com/spreadsheets/d/1g3A679ed7p6Q23rRea6Aav3RPhSiXnrP/edit#gid=1457644454)
dados.rename(columns = {
                     "Hash Rate 30 Day" : "HASH_RATE30",
                     "Active Addresses" : "END_ATIVOS",
                     "Daily Inflation Ratio" : "DY_INFLATION",
                     "Annual Inflation Rate" : "AN_INFLATION",
                     "Median Transaction Size" : "FEE_MEAN",
                     #HODL_W
                     "daysTillHalving" : "DAYS_HALVING",
                     "DELTA_STF" : "DELTA_STF",
    
                     "Pesquisas" : "TRENDS",
                     "sopr" : "SOPR",
                     #LTH
                     "net_unrealized_profit_loss" : "NUPL",
                     "cdd" : "CDD",
    
                     "MayerMutiple" : "MMULT",
                     "MVRV" : "MVRV",
                     "MVRVZ" : "MVRV_Z",
                     "Thermocap" : "THERMOCAP",
                     "DeltaGradient28" : "MRGO28",
                     "DeltaGradient140" : "MRGO140",
                     "BPT" : "BPT"
                     }, inplace = True)

# Apagando indicadores excluídos
dados.drop(["BPT", "DY_INFLATION"], axis = 1, inplace = True)

In [108]:
dados.columns

Index(['AN_INFLATION', 'END_ATIVOS', 'HASH_RATE30', 'FEE_MEAN', 'MMULT',
       'MRGO140', 'MRGO28', 'MVRV', 'MVRV_Z', 'THERMOCAP', 'SPX', 'DXY', 'GLD',
       '10YIELD', 'TRENDS', 'SOPR', 'NUPL', 'CDD', 'DAYS_HALVING',
       'DELTA_STF'],
      dtype='object')

In [112]:
dados.describe().T

,count,mean,std,min,25%,50%,75%,max
AN_INFLATION,4987.0,7.112422e+01,7.914376e+02,7.059107e-01,3.755810e+00,9.097482e+00,2.911240e+01,3.650000e+04
END_ATIVOS,4993.0,4.199272e+05,3.777737e+05,0.000000e+00,2.879900e+04,3.576350e+05,7.411840e+05,1.366494e+06
HASH_RATE30,4958.0,3.661235e+07,5.991494e+07,2.119314e-06,1.294414e+01,4.605822e+05,4.996068e+07,2.216993e+08
FEE_MEAN,4430.0,5.154456e+01,6.083544e+01,8.203471e-08,4.604806e+00,2.156920e+01,8.751409e+01,5.348536e+02
MMULT,4002.0,1.423861e+00,9.927369e-01,2.258761e-01,9.043892e-01,1.169336e+00,1.582944e+00,1.329691e+01
MRGO140,3924.0,9.179007e-01,2.723995e+00,-7.566771e+00,-5.348684e-01,3.909415e-01,1.856018e+00,1.622829e+01
MRGO28,4036.0,5.725798e-01,2.792712e+00,-1.053165e+01,-6.178498e-01,1.437399e-01,1.176056e+00,2.247685e+01
MVRV,4201.0,2.082196e+00,2.874091e+00,3.868285e-01,1.286646e+00,1.743184e+00,2.410370e+00,1.460383e+02
MVRV_Z,4064.0,1.700410e+00,1.895728e+00,-8.300573e-01,4.694225e-01,1.230927e+00,2.301056e+00,1.254730e+01
THERMOCAP,4064.0,8.085980e-07,7.157282e-07,1.202472e-07,3.280976e-07,5.737692e-07,9.953135e-07,6.385165e-06


In [110]:
dados.head()

,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,SPX,DXY,GLD,10YIELD,TRENDS,SOPR,NUPL,CDD,DAYS_HALVING,DELTA_STF
2009-01-03,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [111]:
dados.to_excel("../wrangled_data/indicadores.xlsx", index_label = "Data")